In [19]:
!pip install --upgrade tensorflow
!pip install "git+https://github.com/DavidLandup0/keras-cv@vit"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/DavidLandup0/keras-cv (to revision vit) to /tmp/pip-req-build-76ui5l8a
  Running command git clone -q https://github.com/DavidLandup0/keras-cv /tmp/pip-req-build-76ui5l8a
  Running command git checkout -b vit --track origin/vit
  Switched to a new branch 'vit'
  Branch 'vit' set up to track remote branch 'vit' from 'origin'.


In [20]:
import math
import sys
import os
import pandas as pd

import tensorflow as tf
import keras_cv

import matplotlib.pyplot as plt
import numpy as np

In [21]:
config_names = {
    # JAX name : KCV name, layer_nums
    "Ti/16": ("ViTTiny16", 12),
    "S/16": ("ViTS16", 12),
    "B/16": ("ViTB16", 12),
    "L/16": ("ViTL16", 24),
    "S/32": ("ViTS32", 12),
    "B/32": ("ViTB32", 12),
}

In [22]:
# Choose model to convert
model_to_convert = list(config_names.items())[0]
model_to_convert

('Ti/16', ('ViT_Tiny_16', 12))

In [23]:
model = eval(
    f"keras_cv.models.{model_to_convert[1][0]}(include_rescaling=False, include_top=True, classes=1000, weights=None, input_shape=(224, 224, 3))"
)

In [24]:
with tf.io.gfile.GFile("gs://vit_models/augreg/index.csv") as f:
    df = pd.read_csv(f)

df.head()

,name,ds,epochs,lr,aug,wd,do,sd,best_val,final_val,...,adapt_ds,adapt_lr,adapt_steps,adapt_resolution,adapt_final_val,adapt_final_test,params,infer_samples_per_sec,filename,adapt_filename
0,Ti/16,i1k,300.0,0.001,light0,0.03,0.1,0.1,0.702544,0.702232,...,imagenet2012,0.030,20000,384,0.755698,0.72874,5790000.0,609.58,Ti_16-i1k-300ep-lr_0.001-aug_light0-wd_0.03-do...,Ti_16-i1k-300ep-lr_0.001-aug_light0-wd_0.03-do...
1,Ti/16,i1k,300.0,0.001,light0,0.03,0.1,0.1,0.702544,0.702232,...,imagenet2012,0.010,20000,384,0.754605,0.72412,5790000.0,609.58,Ti_16-i1k-300ep-lr_0.001-aug_light0-wd_0.03-do...,Ti_16-i1k-300ep-lr_0.001-aug_light0-wd_0.03-do...
2,Ti/16,i1k,300.0,0.001,light0,0.03,0.1,0.1,0.702544,0.702232,...,cifar100,0.030,10000,384,0.836000,0.83380,5790000.0,609.58,Ti_16-i1k-300ep-lr_0.001-aug_light0-wd_0.03-do...,Ti_16-i1k-300ep-lr_0.001-aug_light0-wd_0.03-do...
3,Ti/16,i1k,300.0,0.001,light0,0.03,0.1,0.1,0.702544,0.702232,...,cifar100,0.010,10000,384,0.835000,0.83040,5790000.0,609.58,Ti_16-i1k-300ep-lr_0.001-aug_light0-wd_0.03-do...,Ti_16-i1k-300ep-lr_0.001-aug_light0-wd_0.03-do...
4,Ti/16,i1k,300.0,0.001,light0,0.03,0.1,0.1,0.702544,0.702232,...,cifar100,0.003,10000,384,0.800000,0.79620,5790000.0,609.58,Ti_16-i1k-300ep-lr_0.001-aug_light0-wd_0.03-do...,Ti_16-i1k-300ep-lr_0.001-aug_light0-wd_0.03-do...


In [25]:
model_df = df.query(
    f'ds=="i21k" & adapt_resolution==224 & adapt_ds=="imagenet2012" & name=="{model_to_convert[0]}"'
).sort_values("adapt_final_test", ascending=False)
model_df.head()

,name,ds,epochs,lr,aug,wd,do,sd,best_val,final_val,...,adapt_ds,adapt_lr,adapt_steps,adapt_resolution,adapt_final_val,adapt_final_test,params,infer_samples_per_sec,filename,adapt_filename
32844,Ti/16,i21k,300.0,0.001,none,0.03,0.0,0.0,0.415176,0.414551,...,imagenet2012,0.03,20000,224,0.781299,0.73754,5720000.0,3097.42,Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.03-do_...,Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.03-do_...
32845,Ti/16,i21k,300.0,0.001,none,0.03,0.0,0.0,0.415176,0.414551,...,imagenet2012,0.01,20000,224,0.774586,0.73410,5720000.0,3097.42,Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.03-do_...,Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.03-do_...
32981,Ti/16,i21k,300.0,0.001,none,0.10,0.0,0.0,0.412598,0.412529,...,imagenet2012,0.01,20000,224,0.772635,0.73368,5720000.0,3097.42,Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.1-do_0...,Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.1-do_0...
10302,Ti/16,i21k,300.0,0.001,light1,0.03,0.0,0.0,0.397148,0.396895,...,imagenet2012,0.03,20000,224,0.767093,0.73106,5720000.0,3097.42,Ti_16-i21k-300ep-lr_0.001-aug_light1-wd_0.03-d...,Ti_16-i21k-300ep-lr_0.001-aug_light1-wd_0.03-d...
9826,Ti/16,i21k,300.0,0.001,light0,0.03,0.0,0.0,0.407559,0.407314,...,imagenet2012,0.03,20000,224,0.768810,0.73038,5720000.0,3097.42,Ti_16-i21k-300ep-lr_0.001-aug_light0-wd_0.03-d...,Ti_16-i21k-300ep-lr_0.001-aug_light0-wd_0.03-d...


In [26]:
best_model_i1k_checkpoint = str(model_df.iloc[0]["adapt_filename"])
model_df.iloc[0]["adapt_filename"], model_df.iloc[0]["adapt_final_test"]

('Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.03-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.03-res_224',
 0.73754)

In [27]:
filename = best_model_i1k_checkpoint

path = f"gs://vit_models/augreg/{filename}.npz"

print(f"{tf.io.gfile.stat(path).length / 1024 / 1024:.1f} MiB - {path}")

21.9 MiB - gs://vit_models/augreg/Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.03-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.03-res_224.npz


In [28]:
local_path = path.split("//")[-1].split("/")[-1]
local_path

'Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.03-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.03-res_224.npz'

In [29]:
!gsutil cp {path} .

Copying gs://vit_models/augreg/Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.03-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.03-res_224.npz...
| [1 files][ 21.9 MiB/ 21.9 MiB]                                                
Operation completed over 1 objects/21.9 MiB.                                     


In [30]:
with open(local_path, "rb") as f:
    params_jax = np.load(f)
    params_jax = dict(zip(params_jax.keys(), params_jax.values()))

In [31]:
from pprint import pformat

print(pformat(list(params_jax.keys())))

['Transformer/encoder_norm/bias',
 'Transformer/encoder_norm/scale',
 'Transformer/encoderblock_0/LayerNorm_0/bias',
 'Transformer/encoderblock_0/LayerNorm_0/scale',
 'Transformer/encoderblock_0/LayerNorm_2/bias',
 'Transformer/encoderblock_0/LayerNorm_2/scale',
 'Transformer/encoderblock_0/MlpBlock_3/Dense_0/bias',
 'Transformer/encoderblock_0/MlpBlock_3/Dense_0/kernel',
 'Transformer/encoderblock_0/MlpBlock_3/Dense_1/bias',
 'Transformer/encoderblock_0/MlpBlock_3/Dense_1/kernel',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/key/bias',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/key/kernel',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/out/bias',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/out/kernel',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/query/bias',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/query/kernel',
 'Transformer/encoderblock_0/MultiHeadDotProductAttention_1/value/bias',
 'Tr

In [32]:
jax_params_to_kcv_params = {
    "Transformer/posembed_input/pos_embedding": "patch_embedding/embedding/embeddings",
    "embedding/bias": "patch_embedding_1/dense_26/bias",
    "embedding/kernel": "patch_embedding_1/dense_26/kernel",
    "cls": "patch_embedding_1/class_token",
    "Transformer/encoderblock_0/LayerNorm_0/scale": "transformer_encoder_12/layer_normalization_25/gamma",
    "Transformer/encoderblock_0/LayerNorm_0/bias": "transformer_encoder_12/layer_normalization_25/beta",
    "Transformer/encoderblock_0/LayerNorm_2/scale": "transformer_encoder_12/layer_normalization_26/gamma",
    "Transformer/encoderblock_0/LayerNorm_2/bias": "transformer_encoder_12/layer_normalization_26/beta",
    "Transformer/encoderblock_0/MultiHeadDotProductAttention_1/query/kernel": "transformer_encoder_12/multi_head_attention_12/query/kernel",
    "Transformer/encoderblock_0/MultiHeadDotProductAttention_1/query/bias": "transformer_encoder_12/multi_head_attention_12/query/bias",
    "Transformer/encoderblock_0/MultiHeadDotProductAttention_1/key/kernel": "transformer_encoder_12/multi_head_attention_12/key/kernel",
    "Transformer/encoderblock_0/MultiHeadDotProductAttention_1/key/bias": "transformer_encoder_12/multi_head_attention_12/key/bias",
    "Transformer/encoderblock_0/MultiHeadDotProductAttention_1/value/kernel": "transformer_encoder_12/multi_head_attention_12/value/kernel",
    "Transformer/encoderblock_0/MultiHeadDotProductAttention_1/value/bias": "transformer_encoder_12/multi_head_attention_12/value/bias",
    "Transformer/encoderblock_0/MultiHeadDotProductAttention_1/out/kernel": "transformer_encoder_12/multi_head_attention_12/attention_output/kernel",
    "Transformer/encoderblock_0/MultiHeadDotProductAttention_1/out/bias": "transformer_encoder_12/multi_head_attention_12/attention_output/bias",
    "Transformer/encoderblock_0/MlpBlock_3/Dense_0/kernel": "transformer_encoder_12/dense_27/kernel",
    "Transformer/encoderblock_0/MlpBlock_3/Dense_0/bias": "transformer_encoder_12/dense_27/bias",
    "Transformer/encoderblock_0/MlpBlock_3/Dense_1/kernel": "transformer_encoder_12/dense_28/kernel",
    "Transformer/encoderblock_0/MlpBlock_3/Dense_1/bias": "transformer_encoder_12/dense_28/bias",
    # ... other transformer blocks
    "Transformer/encoder_norm/scale": "layer_normalization_49/gamma",
    "Transformer/encoder_norm/bias": "layer_normalization_49/beta",
}

In [33]:
model.layers

In [34]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 patching_and_embedding_1 (P  (None, 197, 192)         185664    
 atchingAndEmbedding)                                            
                                                                 
 dropout_1 (Dropout)         (None, 197, 192)          0         
                                                                 
 transformer_encoder_12 (Tra  (None, 197, 192)         444864    
 nsformerEncoder)                                                
                                                                 
 transformer_encoder_13 (Tra  (None, 197, 192)         444864    
 nsformerEncoder)                                                
                                                           

In [35]:
# Check shapes for the class token and embedding layers
print(params_jax["cls"].shape)
print(params_jax["embedding/kernel"].shape)
print(params_jax["embedding/bias"].shape)
print(params_jax["Transformer/posembed_input/pos_embedding"].shape)

for w in model.layers[1].weights:
    print(w.name, w.shape)

(1, 1, 192)
(16, 16, 3, 192)
(192,)
(1, 197, 192)
patching_and_embedding_1/class_token:0 (1, 1, 192)
patching_and_embedding_1/conv2d_1/kernel:0 (16, 16, 3, 192)
patching_and_embedding_1/conv2d_1/bias:0 (192,)
patching_and_embedding_1/embedding/embeddings:0 (197, 192)


In [36]:
# Copy PatchingAndEmbedding layer
model.layers[1].weights[0].assign(tf.Variable(params_jax["cls"]))
model.layers[1].weights[1].assign(tf.Variable(params_jax["embedding/kernel"]))
model.layers[1].weights[2].assign(tf.Variable(params_jax["embedding/bias"]))
model.layers[1].weights[3].assign(
    tf.Variable(params_jax["Transformer/posembed_input/pos_embedding"].squeeze())
)

<tf.Variable 'UnreadVariable' shape=(197, 192) dtype=float32, numpy=
array([[-0.34854868, -0.02977945, -0.07389098, ...,  0.20677236,
        -0.01907007,  0.02419145],
       [ 0.35433733,  0.07774402, -0.3533324 , ..., -0.00687748,
         0.01606411, -0.05084733],
       [ 0.43602812,  0.11496755,  0.33741263, ..., -0.01119205,
        -0.02812277, -0.02334888],
       ...,
       [-0.7694909 , -0.07064533, -0.03692904, ...,  0.07896642,
         0.0200146 , -0.04633933],
       [-0.5500917 , -0.03244696,  0.06329501, ...,  0.10454354,
         0.00393277, -0.03529308],
       [-0.4606193 ,  0.03806652, -0.8069602 , ...,  0.13817342,
         0.07979445, -0.00859057]], dtype=float32)>

In [37]:
# Check transformer block shapes between JAX and KCV
print(params_jax["Transformer/encoderblock_4/LayerNorm_0/scale"].shape)
print(params_jax["Transformer/encoderblock_4/LayerNorm_0/bias"].shape)
print(params_jax["Transformer/encoderblock_4/LayerNorm_2/scale"].shape)
print(params_jax[f"Transformer/encoderblock_4/LayerNorm_2/bias"].shape)
print(
    params_jax[
        f"Transformer/encoderblock_4/MultiHeadDotProductAttention_1/query/kernel"
    ].shape
)
print(
    params_jax[
        f"Transformer/encoderblock_4/MultiHeadDotProductAttention_1/query/bias"
    ].shape
)
print(
    params_jax[
        f"Transformer/encoderblock_4/MultiHeadDotProductAttention_1/key/kernel"
    ].shape
)
print(
    params_jax[
        f"Transformer/encoderblock_4/MultiHeadDotProductAttention_1/key/bias"
    ].shape
)
print(
    params_jax[
        f"Transformer/encoderblock_4/MultiHeadDotProductAttention_1/value/kernel"
    ].shape
)
print(
    params_jax[
        f"Transformer/encoderblock_4/MultiHeadDotProductAttention_1/value/bias"
    ].shape
)
print(
    params_jax[
        f"Transformer/encoderblock_4/MultiHeadDotProductAttention_1/out/kernel"
    ].shape
)
print(
    params_jax[
        f"Transformer/encoderblock_4/MultiHeadDotProductAttention_1/out/bias"
    ].shape
)
print(params_jax[f"Transformer/encoderblock_4/MlpBlock_3/Dense_0/kernel"].shape)
print(params_jax[f"Transformer/encoderblock_4/MlpBlock_3/Dense_0/bias"].shape)
print(params_jax[f"Transformer/encoderblock_4/MlpBlock_3/Dense_1/kernel"].shape)
print(params_jax[f"Transformer/encoderblock_4/MlpBlock_3/Dense_1/bias"].shape)

for w in model.layers[4].weights:
    print(w.name, w.shape)

(192,)
(192,)
(192,)
(192,)
(192, 3, 64)
(3, 64)
(192, 3, 64)
(3, 64)
(192, 3, 64)
(3, 64)
(3, 64, 192)
(192,)
(192, 768)
(768,)
(768, 192)
(192,)
transformer_encoder_13/layer_normalization_27/gamma:0 (192,)
transformer_encoder_13/layer_normalization_27/beta:0 (192,)
transformer_encoder_13/layer_normalization_28/gamma:0 (192,)
transformer_encoder_13/layer_normalization_28/beta:0 (192,)
transformer_encoder_13/multi_head_attention_13/query/kernel:0 (192, 3, 64)
transformer_encoder_13/multi_head_attention_13/query/bias:0 (3, 64)
transformer_encoder_13/multi_head_attention_13/key/kernel:0 (192, 3, 64)
transformer_encoder_13/multi_head_attention_13/key/bias:0 (3, 64)
transformer_encoder_13/multi_head_attention_13/value/kernel:0 (192, 3, 64)
transformer_encoder_13/multi_head_attention_13/value/bias:0 (3, 64)
transformer_encoder_13/multi_head_attention_13/attention_output/kernel:0 (3, 64, 192)
transformer_encoder_13/multi_head_attention_13/attention_output/bias:0 (192,)
transformer_encoder_13

In [38]:
# Copy Transformer Encoders
for i in range(model_to_convert[1][1]):
    model.layers[3 + i].weights[0].assign(
        tf.Variable(params_jax[f"Transformer/encoderblock_{i}/LayerNorm_0/scale"])
    )
    model.layers[3 + i].weights[1].assign(
        tf.Variable(params_jax[f"Transformer/encoderblock_{i}/LayerNorm_0/bias"])
    )
    model.layers[3 + i].weights[2].assign(
        tf.Variable(params_jax[f"Transformer/encoderblock_{i}/LayerNorm_2/scale"])
    )
    model.layers[3 + i].weights[3].assign(
        tf.Variable(params_jax[f"Transformer/encoderblock_{i}/LayerNorm_2/bias"])
    )
    model.layers[3 + i].weights[4].assign(
        tf.Variable(
            params_jax[
                f"Transformer/encoderblock_{i}/MultiHeadDotProductAttention_1/query/kernel"
            ]
        )
    )
    model.layers[3 + i].weights[5].assign(
        tf.Variable(
            params_jax[
                f"Transformer/encoderblock_{i}/MultiHeadDotProductAttention_1/query/bias"
            ]
        )
    )
    model.layers[3 + i].weights[6].assign(
        tf.Variable(
            params_jax[
                f"Transformer/encoderblock_{i}/MultiHeadDotProductAttention_1/key/kernel"
            ]
        )
    )
    model.layers[3 + i].weights[7].assign(
        tf.Variable(
            params_jax[
                f"Transformer/encoderblock_{i}/MultiHeadDotProductAttention_1/key/bias"
            ]
        )
    )
    model.layers[3 + i].weights[8].assign(
        tf.Variable(
            params_jax[
                f"Transformer/encoderblock_{i}/MultiHeadDotProductAttention_1/value/kernel"
            ]
        )
    )
    model.layers[3 + i].weights[9].assign(
        tf.Variable(
            params_jax[
                f"Transformer/encoderblock_{i}/MultiHeadDotProductAttention_1/value/bias"
            ]
        )
    )
    model.layers[3 + i].weights[10].assign(
        tf.Variable(
            params_jax[
                f"Transformer/encoderblock_{i}/MultiHeadDotProductAttention_1/out/kernel"
            ]
        )
    )
    model.layers[3 + i].weights[11].assign(
        tf.Variable(
            params_jax[
                f"Transformer/encoderblock_{i}/MultiHeadDotProductAttention_1/out/bias"
            ].reshape(model.layers[3 + i].weights[11].shape)
        )
    )
    model.layers[3 + i].weights[12].assign(
        tf.Variable(
            params_jax[f"Transformer/encoderblock_{i}/MlpBlock_3/Dense_0/kernel"]
        )
    )
    model.layers[3 + i].weights[13].assign(
        tf.Variable(params_jax[f"Transformer/encoderblock_{i}/MlpBlock_3/Dense_0/bias"])
    )
    model.layers[3 + i].weights[14].assign(
        tf.Variable(
            params_jax[f"Transformer/encoderblock_{i}/MlpBlock_3/Dense_1/kernel"]
        )
    )
    model.layers[3 + i].weights[15].assign(
        tf.Variable(params_jax[f"Transformer/encoderblock_{i}/MlpBlock_3/Dense_1/bias"])
    )

In [39]:
print(params_jax["Transformer/encoder_norm/scale"].shape)
print(params_jax["Transformer/encoder_norm/bias"].shape)

for w in model.layers[15].weights:
    print(w.name, w.shape)

(192,)
(192,)
layer_normalization_49/gamma:0 (192,)
layer_normalization_49/beta:0 (192,)


In [40]:
# Copy layer norm before class head
model.layers[15].weights[0].assign(
    tf.Variable(params_jax["Transformer/encoder_norm/scale"])
)
model.layers[15].weights[1].assign(
    tf.Variable(params_jax["Transformer/encoder_norm/bias"])
)

<tf.Variable 'UnreadVariable' shape=(192,) dtype=float32, numpy=
array([ 2.8223321e+00,  8.7457395e-01, -3.7481210e-01,  1.3750441e+00,
        4.1197315e-02,  4.8303711e-01, -2.6310286e-01,  3.6670661e-01,
       -2.1512895e+00,  2.2048175e+00, -1.3401504e+00,  2.3340700e-02,
        8.6943895e-01, -2.1459769e-01,  5.6879807e-01,  9.7927111e-01,
        4.7838250e-01, -3.2664791e-01, -1.1942422e-01, -1.3508632e+00,
       -1.8508923e-01, -2.8242302e+00,  1.5803963e+00,  4.2268095e+00,
       -4.0262440e-01,  6.7515630e-01, -9.2384642e-01, -6.8262619e-01,
       -7.9978913e-02,  5.8783874e-02, -1.8007267e+00, -4.0809026e-01,
        1.0480539e+00,  2.2342615e+00,  2.4652045e+00,  1.9197918e-01,
        1.3735850e+00, -4.0193820e-01,  5.7247961e-01,  6.2033898e-01,
       -1.1311350e+00,  7.4566154e+00, -1.1646005e+00,  1.2841954e+00,
       -4.0062070e-01,  1.6707023e+00, -1.7686105e+00, -3.9713845e-01,
        9.1295403e-01, -1.7427791e+00, -7.2540677e-01, -9.1626835e-01,
        3.30

In [41]:
print(params_jax["head/kernel"].shape)
print(params_jax["head/bias"].shape)

for w in model.layers[17].weights:
    print(w.name, w.shape)

(192, 1000)
(1000,)
dense_49/kernel:0 (192, 1000)
dense_49/bias:0 (1000,)


In [42]:
# Copy haed kernel and bias
model.layers[17].weights[0].assign(tf.Variable(params_jax["head/kernel"]))
model.layers[17].weights[1].assign(tf.Variable(params_jax["head/bias"]))

<tf.Variable 'UnreadVariable' shape=(1000,) dtype=float32, numpy=
array([-3.12163169e-03,  7.99638146e-05, -4.45322460e-03, -5.63935703e-03,
       -5.19127958e-03,  1.89294806e-03, -3.09978402e-03,  2.00790493e-03,
        1.12810067e-03, -4.83769877e-03, -2.97148316e-03, -6.80502411e-03,
       -2.76475097e-03, -5.29246219e-03, -5.48043428e-03, -4.45886562e-03,
       -1.57492165e-03, -2.84638233e-03, -9.20321385e-04, -5.74108539e-03,
       -5.28989173e-03,  2.09719199e-03, -5.66764874e-03,  1.98722444e-03,
       -5.97396633e-03, -2.90608662e-03,  1.00772153e-03, -4.28308034e-03,
       -1.85368024e-03,  6.58091565e-04, -5.24879014e-03, -3.40827229e-03,
        2.74766865e-03, -1.61942351e-03, -1.62760960e-04, -2.39770347e-03,
        9.26783402e-03, -4.49899398e-03, -5.08973782e-04,  1.13704715e-04,
       -4.74586571e-03,  8.98223298e-05, -3.58532229e-03, -1.97856897e-03,
        3.77449498e-04, -2.17942358e-03, -3.71574331e-03,  1.31800561e-03,
       -8.61194357e-03, -1.9997034

In [43]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

import PIL
import urllib


def url_to_array(url):
    req = urllib.request.urlopen(url)
    arr = np.array(bytearray(req.read()), dtype=np.int8)
    arr = cv2.imdecode(arr, -1)
    arr = cv2.cvtColor(arr, cv2.COLOR_BGR2RGB)
    arr = cv2.resize(arr, (224, 224))
    return arr


def preprocess_image(image, label):
    image_resized = tf.image.resize(image, (224, 224))
    image_resized = tf.cast(image_resized, tf.float32)
    image_resized = (image_resized - 127.5) / 127.5
    return image_resized, label

In [44]:
cat = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/Cat_November_2010-1a.jpg/1200px-Cat_November_2010-1a.jpg"
cat_img = url_to_array(cat)
cat_img, _ = preprocess_image(cat_img, None)
cat_img = tf.expand_dims(cat_img, 0)

In [45]:
!wget https://storage.googleapis.com/bit_models/ilsvrc2012_wordnet_lemmas.txt -O ilsvrc2012_wordnet_lemmas.txt

with open("ilsvrc2012_wordnet_lemmas.txt", "r") as f:
    lines = f.readlines()
imagenet_int_to_str = [line.rstrip() for line in lines]

--2022-12-02 02:18:51--  https://storage.googleapis.com/bit_models/ilsvrc2012_wordnet_lemmas.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.99.128, 173.194.202.128, 173.194.203.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.99.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21675 (21K) [text/plain]
Saving to: ‘ilsvrc2012_wordnet_lemmas.txt’

ilsvrc2012_wordnet_ 100%[===================>]  21.17K  --.-KB/s    in 0s      

2022-12-02 02:18:52 (89.2 MB/s) - ‘ilsvrc2012_wordnet_lemmas.txt’ saved [21675/21675]



In [46]:
predictions = model.predict(cat_img)

1/1 [==============================] - 13s 13s/step


In [47]:
top_5 = tf.math.top_k(predictions, k=5, sorted=False)
top_5

TopKV2(values=<tf.Tensor: shape=(1, 5), dtype=float32, numpy=
array([[0.51381004, 0.33035758, 0.11933842, 0.02861441, 0.00126063]],
      dtype=float32)>, indices=<tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[281, 282, 285, 287, 288]], dtype=int32)>)

In [48]:
pred = np.argmax(predictions)
imagenet_int_to_str[int(pred)]

'tabby, tabby_cat'

In [49]:
dog_url = "https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/dog-puppy-on-garden-royalty-free-image-1586966191.jpg?crop=1.00xw:0.669xh;0,0.190xh&resize=640:*"
dog_img = url_to_array(dog_url)
dog_img, _ = preprocess_image(dog_img, None)
dog_img = tf.expand_dims(dog_img, 0)
predictions = model.predict(dog_img)
pred = np.argmax(predictions)
imagenet_int_to_str[int(pred)]

1/1 [==============================] - 0s 110ms/step


'golden_retriever'

In [50]:
model.compile(
    "adam",
    "sparse_categorical_crossentropy",
    metrics=["accuracy", tf.keras.metrics.SparseTopKCategoricalAccuracy(5)],
)

In [51]:
import tensorflow_datasets as tfds

(test_set), info = tfds.load(
    "imagenet_v2", split=["test"], as_supervised=True, with_info=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imagenet_v2/matched-frequency/3.0.0.incomplete39QMVU/imagenet_v2-test.tfrecord…

Dataset imagenet_v2 downloaded and prepared to ~/tensorflow_datasets/imagenet_v2/matched-frequency/3.0.0. Subsequent calls will reuse this data.


In [52]:
test_set = (
    test_set[0]
    .shuffle(len(test_set[0]))
    .map(preprocess_image)
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)

In [53]:
for entry, label in test_set.take(1):
    print(label)

tf.Tensor(
[750 231 633 730 465 399 316 242 157 466 537 704 311 974 763 132 281 885
 900 372 444 221 727 330 254  87 950 307 563 690 544  80], shape=(32,), dtype=int64)


In [54]:
model.evaluate(test_set)

313/313 [==============================] - 58s 89ms/step - loss: 1.6895 - accuracy: 0.6067 - sparse_top_k_categorical_accuracy: 0.8285


[1.6895182132720947, 0.6067000031471252, 0.828499972820282]

In [55]:
model.save(f"{model_to_convert[1][0]}.h5")